In [ ]:
# TODO(@lpupp)
# [ ] save stuff: y_hats, models, (weights) sgd path
# [ ] are there other ways to code LOO that might be faster?
# [ ] initialization questions (should we use same seed to initialize all networks or do we like the randomness?)
# [X] fix bugs in code and make sure it runs
# [ ] does everything do what I think it does?
# [ ] put run_train and leave_one_out somewhere useful
# [ ] generate data (and source)
# [ ] run simulation with small data set

# Questions
# [ ] initialization: should we use same seed to initialize all networks or do we like the randomness?
# [ ] are we validating?
# [ ] in what format should the weights be saved? table?

In [1]:
from sklearn.model_selection import LeaveOneOut
import numpy as np

X = np.array([["a"], ["b"], ["c"], ["d"], ["e"], ["f"]])
loo = LeaveOneOut()
loo.get_n_splits(X)
for train_index, test_index in loo.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    #y_train, y_test = y[train_index], y[test_index]
    #print(X_train, X_test, y_train, y_test)
    print(X_train, X_test)

TRAIN: [1 2 3 4 5] TEST: [0]
[['b']
 ['c']
 ['d']
 ['e']
 ['f']] [['a']]
TRAIN: [0 2 3 4 5] TEST: [1]
[['a']
 ['c']
 ['d']
 ['e']
 ['f']] [['b']]
TRAIN: [0 1 3 4 5] TEST: [2]
[['a']
 ['b']
 ['d']
 ['e']
 ['f']] [['c']]
TRAIN: [0 1 2 4 5] TEST: [3]
[['a']
 ['b']
 ['c']
 ['e']
 ['f']] [['d']]
TRAIN: [0 1 2 3 5] TEST: [4]
[['a']
 ['b']
 ['c']
 ['d']
 ['f']] [['e']]
TRAIN: [0 1 2 3 4] TEST: [5]
[['a']
 ['b']
 ['c']
 ['d']
 ['e']] [['f']]


# LOONNet training script

In [37]:
import numpy as np
from sklearn.model_selection import LeaveOneOut, train_test_split
from keras.utils.np_utils import to_categorical   

import tensorflow as tf
import time

import utils # from *this directory* -- some helper functions

Using TensorFlow backend.


In [3]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 100
#n_train = 60000
#n_test = 10000

#n_pixels = 28

In [79]:
# Step 1: Read in data
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

# One-hot labels
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Normalize
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

# Flatten
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1] * x_train.shape[2]))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1] * x_test.shape[2]))

# Validation split
#x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.66, random_state=1, shuffle=True)

In [80]:
# Step 2: Placeholders for x and y
x = tf.placeholder(tf.float32, shape=(batch_size, x_train.shape[1]))
y = tf.placeholder(tf.float32, shape=(batch_size, y_train.shape[1]))

In [81]:
print('shapes:')
print('x_train: {}'.format(x_train.shape))
#print('x_val: {}'.format(x_val.shape))
print('x_test: {}'.format(x_test.shape))
print('y_train: {}'.format(y_train.shape))

shapes:
x_train: (60000, 784)
x_test: (10000, 784)
y_train: (60000, 10)


Notes:

    * `tf.data.Dataset.from_tensor_slices()` - create tensors from data in memory
    * `.batch` - Combines consecutive elements of this dataset into batches
    * `tf.data.Iterator`provides the main way to extract elements from a dataset. The operation returned by Iterator.get_next() yields the next element of a Dataset when executed, and typically acts as the interface between input pipeline code and your model.
    * 

In [99]:
# Step 3: create weights and bias
# w is initialized to random variables with mean of 0, stddev of 0.01
# b is initialized to 0
# shape of w depends on the dimension of X and Y so that Y = tf.matmul(X, w)
# shape of b depends on Y
n_input = x_train.shape[1]
n_hidden_1 = 32
n_classes = y_train.shape[1]

tf.reset_default_graph()

# Difference between `get_variable` and `Variable`:
# `Variable` is low-level, `get_variable` is higher level with more functionality
w_h1 = tf.get_variable(name='weights_h1', shape=(n_input, n_hidden_1), initializer=tf.random_normal_initializer(0, 0.01))
b_h1 = tf.get_variable(name='bias_h1', shape=(1, n_hidden_1), initializer=tf.zeros_initializer())

w_out = tf.get_variable(name='weights_h2', shape=(n_hidden_1, n_classes), initializer=tf.random_normal_initializer(0, 0.01))
b_out = tf.get_variable(name='bias_h2', shape=(1, n_classes), initializer=tf.zeros_initializer())


In [103]:
# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
layer_1 = tf.add(tf.matmul(x, w_h1), b_h1) # TODO(lpupp)
layer_1 = tf.nn.relu(layer_1)
logits = tf.add(tf.matmul(layer_1, w_out), b_out)

In [104]:
# Step 5: define loss function
# use cross entropy of softmax of logits as the loss function
# note the original function without `_v2` is deprecated so we should transition
entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y, name='entropy')
loss = tf.reduce_mean(entropy, name='loss') # computes the mean over all the examples in the batch

In [105]:
# Step 6: define training op
# using gradient descent with learning rate of 0.01 to minimize loss
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [106]:
# Step 7: calculate accuracy with test set
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

In [113]:
# https://stackoverflow.com/questions/38164798/does-tensorflow-have-cross-validation-implemented-for-its-users?rq=1
def run_train(sess, x_train, y_train, save_weights=False):
    print("\nStart training")
    if save_weights:
        w = np.zeros((n_train, n_train, 1)) # TODO(@lpupp) fix size to 
    
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epochs):
        total_loss = 0
        n_batches = 0
        total_batch = int(x_train.shape[0] / batch_size)
        
        for i in range(total_batch):
            x_batch = x_train[i*batch_size:(i+1)*batch_size]
            y_batch = y_train[i*batch_size:(i+1)*batch_size]
            _, l = sess.run([optimizer, loss], feed_dict={x: x_batch, y: y_batch})
            total_loss += l
            n_batches += 1
        if epoch % 50 == 0:
            print('Average loss epoch {0}: {1}'.format(epoch, total_loss/n_batches))
        
        # TODO(@lpupp) Save stuff (SGD path)
        # TODO(@lpupp) Turn this into function and stick it in utilsß
        if save_weights:
            # flatten weights, append, and save (?)
            sess.run(w_h1) 

def leave_one_out(sess#, 
                  #saver, 
                  #filepath
                 ):
    yhats = np.zeros((n_train, n_train, 1))
    
    loo = LeaveOneOut()
    loo_idx = 0
    
    sess.run(train_init)  # TODO(@lpupp) fix for below setup with (train_data, train_y_all)
    for train_idx, _ in loo.split(x_train, y_train):
        train_x = x_train[train_idx]
        train_y = y_train[train_idx]
        #val_x = x_train[_]
        #val_y = y_train[_]

        single_model_start_time = time.time()
        print('Training LOO model {0}'.format(loo_idx))
        run_train(sess, x_train, y_train)
        print('Model {0} time: {1} seconds'.format(loo_idx, time.time() - single_model_start_time))
        
        # TODO(@lpupp) Save stuff (model, y_hats)
        #yhats[:, loo_idx] = sess.run(test_init)
        #save_path = saver.save(sess, '/' + filepath + '.ckpt')
        
        loo_idx += 1
        
    return results

In [114]:
# Run model
#saver = tf.train.Saver()
#_filepath = None
train_init = tf.global_variables_initializer()

with tf.Session() as sess:
    total_start_time = time.time()
    
    result = leave_one_out(sess) #,saver, filepath) 
    print('Total time: {0} seconds'.format(time.time() - total_start_time))
    print("leave one out result: %s" % result)
    
    # test the model
    sess.run(test_init) # drawing samples from test_data
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass

    print('Accuracy {0}'.format(total_correct_preds/n_test)) # TODO(@lpupp) where does n_test come from?

Training LOO model 0

Start training
Average loss epoch 0: 2.2790440590972576
Average loss epoch 50: 0.20502612154739788


KeyboardInterrupt: 